In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

# WordCloud and matplotlib for visualization
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## TO DO

- assigning keywords to missing keyword labels - DONE
- remove symbols (#, ".. - DONE
- remove links - DONE
- validation
- make slides - IN PROCESS
- TFIDF, LSA, LSTM / RNNs, the list is long
- try different classifiers
- reading discussion board for inspiration


In [2]:
train = pd.read_csv("preprocessed_train_data.csv", index_col = 0)
test = pd.read_csv("preprocessed_test_data.csv", index_col = 0)

trained_tweets = train['keyword']+train['text']
test_tweets = test['keyword']+test['text']

In [3]:
trained_tweets.head()

0    earthquak  deed reason earthquak may allah for...
1     forest fire  forest fire near la rang ask canada
2    evacu  resid ask shelter place notifi offic ev...
3    wildfir  peopl receiv wildfir evacu order cali...
4    wildfir  got sent photo rubi alaska smoke wild...
dtype: object

In [4]:
test_tweets.head()

0                      crash  happen terribl car crash
1    earthquak  heard earthquak differ citi stay sa...
2    forest fire  forest fire spot pond gees flee a...
3            apocalyps  apocalyps light spokan wildfir
4          typhoon  typhoon soudelor kill china taiwan
dtype: object

# Encoding and Vectorizers

In [5]:
# import category_encoders as ce

# # Target encoding
# features = ['keyword']
# encoder = ce.TargetEncoder(cols=features)
# encoder.fit(train[features],train['target'])

# train = train.join(encoder.transform(train[features]).add_suffix('_target'))
# test = test.join(encoder.transform(test[features]).add_suffix('_target'))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(trained_tweets, train['target'].values, 
        train_size = 0.80, test_size = 0.20, random_state = 12, shuffle=True) # We have split train-test dataset using 80:20 ratio

# TFIDF

In [7]:
#precompute vectorized representations
word_vectorizer = TfidfVectorizer(
    analyzer='word',
    stop_words='english',
    ngram_range=(1, 3),
    lowercase=True,
    min_df=5,
    max_features=30000)

char_vectorizer = TfidfVectorizer(
    analyzer='char',
    stop_words='english',
    ngram_range=(3, 6),
    lowercase=True,
    min_df=5,
    max_features=50000)

vectorizer = FeatureUnion([('word_vectorizer', word_vectorizer),  ('char_vectorizer', char_vectorizer)])
vectorizer.fit(X_train)

X_train_vectors = vectorizer.transform(X_train).toarray()
X_test_vectors = vectorizer.transform(X_test).toarray()
print(X_train_vectors.shape, X_test_vectors.shape)

#X_train_text = X_train.tolist()
#X_test_text = X_test.tolist()

(6090, 48864) (1523, 48864)


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

## Tokenize the sentences
max_features=50000
maxlen=25
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train.tolist())
X_train = tokenizer.texts_to_sequences(X_train.tolist())
X_test = tokenizer.texts_to_sequences(X_test.tolist())

## Pad the sentences 
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

Using TensorFlow backend.


In [9]:
X_train

array([[   0,    0,    0, ...,  353,  716,  838],
       [   0,    0,    0, ...,   39,  283,  403],
       [   0,    0,    0, ..., 1614,    9, 2313],
       ...,
       [   0,    0,    0, ..., 1496,  947,  310],
       [   0,    0,    0, ...,  364,  513, 1019],
       [   0,    0,    0, ...,  213,   73,  729]], dtype=int32)

# Recurrent Neural Networks

In [10]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout, Embedding
from tensorflow.keras import backend as K

vocab_dim = X_train.max() + 1
embed_dim = int(np.sqrt(vocab_dim))
print(vocab_dim, embed_dim)
model = Sequential([
    Embedding(input_dim=vocab_dim, output_dim=embed_dim),
    SimpleRNN(units=embed_dim, return_sequences=False),
    Dense(1, activation='sigmoid')
])
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

10401 101


In [11]:
history = model.fit(
    x=X_train, y=y_train, batch_size=64, epochs=5, verbose=True, validation_data=(X_test, y_test), shuffle=True
)
K.clear_session()

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 5s 821us/sample - loss: 0.5847 - accuracy: 0.6849 - val_loss: 0.4948 - val_accuracy: 0.7833
Epoch 2/5
6090/6090 [==============================] - 3s 512us/sample - loss: 0.2561 - accuracy: 0.9046 - val_loss: 0.5635 - val_accuracy: 0.7577
Epoch 3/5
6090/6090 [==============================] - 3s 543us/sample - loss: 0.1218 - accuracy: 0.9624 - val_loss: 0.5828 - val_accuracy: 0.7656
Epoch 4/5
6090/6090 [==============================] - 3s 521us/sample - loss: 0.0820 - accuracy: 0.9739 - val_loss: 0.6205 - val_accuracy: 0.7669
Epoch 5/5
6090/6090 [==============================] - 4s 583us/sample - loss: 0.0653 - accuracy: 0.9793 - val_loss: 0.7195 - val_accuracy: 0.7702


In [12]:
model = Sequential([
    Embedding(input_dim=vocab_dim, output_dim=embed_dim),
    SimpleRNN(units=embed_dim, return_sequences=True),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [13]:
history = model.fit(
    x=X_train, y=y_train, batch_size=64, epochs=5, verbose=True, validation_data=(X_test, y_test), shuffle=True
)
K.clear_session()

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 5s 843us/sample - loss: 0.6048 - accuracy: 0.6813 - val_loss: 0.4801 - val_accuracy: 0.7794
Epoch 2/5
6090/6090 [==============================] - 3s 540us/sample - loss: 0.3515 - accuracy: 0.8576 - val_loss: 0.5017 - val_accuracy: 0.7676
Epoch 3/5
6090/6090 [==============================] - 4s 606us/sample - loss: 0.2208 - accuracy: 0.9228 - val_loss: 0.5292 - val_accuracy: 0.7702
Epoch 4/5
6090/6090 [==============================] - 4s 606us/sample - loss: 0.1462 - accuracy: 0.9529 - val_loss: 0.6095 - val_accuracy: 0.7669
Epoch 5/5
6090/6090 [==============================] - 4s 640us/sample - loss: 0.1032 - accuracy: 0.9693 - val_loss: 0.7132 - val_accuracy: 0.7682


# Bidirectional RNN

In [14]:
from tensorflow.keras.layers import Bidirectional

model = Sequential([
    Embedding(input_dim=vocab_dim, output_dim=embed_dim),
    Bidirectional(SimpleRNN(units=embed_dim, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [15]:
history = model.fit(
    x=X_train, y=y_train, batch_size=64, epochs=5, verbose=True, validation_data=(X_test, y_test), shuffle=True
)
K.clear_session()

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 9s 2ms/sample - loss: 0.5807 - accuracy: 0.7025 - val_loss: 0.4848 - val_accuracy: 0.7958
Epoch 2/5
6090/6090 [==============================] - 6s 962us/sample - loss: 0.3223 - accuracy: 0.8696 - val_loss: 0.4991 - val_accuracy: 0.7866
Epoch 3/5
6090/6090 [==============================] - 5s 896us/sample - loss: 0.1884 - accuracy: 0.9315 - val_loss: 0.5673 - val_accuracy: 0.7800
Epoch 4/5
6090/6090 [==============================] - 6s 928us/sample - loss: 0.1126 - accuracy: 0.9614 - val_loss: 0.6587 - val_accuracy: 0.7400
Epoch 5/5
6090/6090 [==============================] - 6s 975us/sample - loss: 0.0854 - accuracy: 0.9703 - val_loss: 0.6266 - val_accuracy: 0.7663


# LSTM

In [16]:
from tensorflow.keras.layers import LSTM

model = Sequential([
    Embedding(input_dim=vocab_dim, output_dim=embed_dim),
    LSTM(units=embed_dim, return_sequences=False),
    Dense(1, activation='sigmoid')
])
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [17]:
history = model.fit(
    x=X_train, y=y_train, batch_size=64, epochs=5, verbose=True, validation_data=(X_test, y_test), shuffle=True
)
K.clear_session()

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 11s 2ms/sample - loss: 0.5628 - accuracy: 0.7103 - val_loss: 0.4604 - val_accuracy: 0.7925
Epoch 2/5
6090/6090 [==============================] - 8s 1ms/sample - loss: 0.3171 - accuracy: 0.8742 - val_loss: 0.4866 - val_accuracy: 0.7814
Epoch 3/5
6090/6090 [==============================] - 8s 1ms/sample - loss: 0.2075 - accuracy: 0.9232 - val_loss: 0.6113 - val_accuracy: 0.7689
Epoch 4/5
6090/6090 [==============================] - 8s 1ms/sample - loss: 0.1446 - accuracy: 0.9461 - val_loss: 0.6945 - val_accuracy: 0.7728
Epoch 5/5
6090/6090 [==============================] - 8s 1ms/sample - loss: 0.1063 - accuracy: 0.9637 - val_loss: 0.7461 - val_accuracy: 0.7702
